<a href="https://colab.research.google.com/github/OscarW99/applied-ml-series/blob/main/ClassicalML1_2_EndToEnd_BreastCancerClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intro1.2_EndToEnd_BreastCancerClassification

This notebook walks through an end-to-end machine learning project for classifying breast cancer tumors as malignant or benign using the Wisconsin Breast Cancer dataset from Scikit-learn.

# Project Roadmap:
1.  Get the data.
2.  Take a first look (explore & visualize).
3.  Split into training and test sets (super important!).
4.  More exploration on the training set.
5.  Prepare the data for our ML algorithms (cleaning, scaling).
6.  Select and train a few models.
7.  Fine-tune the best model.
8.  Evaluate on the test set.
9.  Briefly touch on saving the model.

---
# 1. Getting Our Hands on the Data

We'll use the Wisconsin Breast Cancer dataset, which is conveniently included in Scikit-Learn. It's a classic dataset for this kind of binary classification task.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer

# Load the dataset
cancer = load_breast_cancer()
print(dir(cancer))
print(cancer.DESCR)
print(cancer.data)


In [ ]:
# Create a Pandas DataFrame
# Features are in cancer.data, target (diagnosis) is in cancer.target
# Feature names are in cancer.feature_names
df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
df.head()

In [ ]:
# The target in scikit-learn's dataset is often 0 for malignant and 1 for benign.
# For clarity and common convention (1 for positive class), let's map:
# Malignant -> 1
# Benign -> 0
df['diagnosis'] = (cancer.target == 0).astype(int) # 0 is malignant in original, so (cancer.target == 0) is True (1) for malignant

print("Dataset loaded successfully.")
print(f"Shape of the dataset: {df.shape}")

# Let's see what `cancer.target` looked like and how our `diagnosis` column is now.
# `cancer.target`: 0 means malignant, 1 means benign.
# Our `df['diagnosis']`: 1 means malignant, 0 means benign.

print("\nOriginal Scikit-learn target unique values:", np.unique(cancer.target))
print("Our 'diagnosis' column unique values:", np.unique(df['diagnosis']))
print("Counts for our 'diagnosis' column:")
print(df['diagnosis'].value_counts())

---
# 2. Initial Data Exploration & Visualization

Let's get a first look at the data structure and feature distributions, especially how they differ by diagnosis.

In [ ]:
print("\nFirst 5 rows of the dataset:")
print(df.head())

print("\nDataset info (columns, data types, non-null counts):")
df.info()

print("\nStatistical summary of numerical features:")
print(df.describe())

Now, let's plot histograms for all numerical features, separated by diagnosis. This will give us a much better idea of which features might be good discriminators.

In [ ]:
# # Plot histograms for a all features
numerical_features = df.drop('diagnosis', axis=1).columns # Get all feature column names

# Create histograms for all numerical features, separated by diagnosis
# We'll plot a few at a time for readability, or you can loop through all.
# Let's determine a reasonable number of rows and columns for subplots
n_features = len(numerical_features)
n_cols = 3 # Or 4, depending on preference
n_rows = (n_features + n_cols - 1) // n_cols # Ceiling division

fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 5, n_rows * 4))
axes = axes.flatten() # Flatten to easily iterate over axes

for i, feature in enumerate(numerical_features):
    sns.histplot(data=df, x=feature, hue='diagnosis', kde=True, ax=axes[i], palette={0: 'skyblue', 1: 'salmon'})
    axes[i].set_title(f'Distribution of {feature}')
    axes[i].legend(title='Diagnosis', labels=['Malignant (1)', 'Benign (0)']) # Adjust labels based on hue order

# Hide any unused subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.suptitle("Feature Distributions by Diagnosis (0=Benign, 1=Malignant)", fontsize=16, y=1.02)
plt.show()

The histograms above show how the values for each feature are distributed for benign (skyblue) versus malignant (salmon) tumors. Features where the two colored distributions show good separation are likely to be good predictors.

---
# 3. Split into Training and Test Sets

This is a critical step! We set aside a portion of the data for final testing and do not touch it during model development or tuning. We use stratification to maintain class proportions.

In [ ]:
from sklearn.model_selection import train_test_split

# Separate features (X) and target (y)
X = df.drop('diagnosis', axis=1)
y = df['diagnosis']

# Stratified split
# test_size=0.2 means 20% for test, 80% for train
# random_state ensures we get the same split every time we run the code
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Training set shape: {X_train.shape}, Test set shape: {X_test.shape}")
print("\nTraining set diagnosis distribution:")
print(y_train.value_counts(normalize=True))
print("\nTest set diagnosis distribution:")
print(y_test.value_counts(normalize=True)) # Should be similar to training set

The distributions look similar, so our stratified split worked well! Now, we lock away `X_test` and `y_test`.

---
# 4. Deeper Exploration & Visualization on the Training Set

Now, we only work with `X_train` and `y_train`.

**UMAP Visualization**

Let's use UMAP to try and visualize the overall structure of our training data in 2D, colored by diagnosis. UMAP works best on scaled data.
(You might need to install UMAP: `!pip install umap-learn`)

In [ ]:
# For UMAP, you'd typically scale the data first
# Make sure you have umap-learn installed: pip install umap-learn
# You can run this in a cell: !pip install umap-learn
try:
    from sklearn.preprocessing import StandardScaler
    from umap import UMAP

    # Scale a copy of X_train for UMAP
    X_train_scaled_for_umap = StandardScaler().fit_transform(X_train)

    reducer = UMAP(n_neighbors=15, min_dist=0.1, n_components=2, random_state=42, n_jobs=1) # n_jobs=1 for reproducibility
    X_train_umap = reducer.fit_transform(X_train_scaled_for_umap)

    plt.figure(figsize=(10, 8))
    # y_train == 0 is Benign, y_train == 1 is Malignant
    plt.scatter(X_train_umap[y_train==0, 0], X_train_umap[y_train==0, 1], label="Benign (0)", c="skyblue", alpha=0.7)
    plt.scatter(X_train_umap[y_train==1, 0], X_train_umap[y_train==1, 1], label="Malignant (1)", c="salmon", alpha=0.7)
    plt.xlabel("UMAP Component 1")
    plt.ylabel("UMAP Component 2")
    plt.title("UMAP of Training Samples (Scaled) by Diagnosis")
    plt.legend()
    plt.show()
except ImportError:
    print("UMAP not installed or an issue occurred. Skipping UMAP plot.")
    print("To install UMAP: !pip install umap-learn (run this in a new code cell if needed)")

The UMAP plot should hopefully show some separation between the benign and malignant clusters. If they are heavily overlapped, the problem might be very hard.

### **Correlation Analysis on Training Data**

Let's see how features correlate with the diagnosis in our training set.

In [ ]:
# Add diagnosis back to training set (temporarily) for correlation calculation
X_train_with_target = X_train.copy()
X_train_with_target['diagnosis'] = y_train

# Calculate the correlation matrix
corr_matrix_train = X_train_with_target.corr()

# Visualize the correlation of features with the 'diagnosis' column using a heatmap
plt.figure(figsize=(8, 12)) # Adjusted for more features potentially
sns.heatmap(corr_matrix_train[['diagnosis']].sort_values(by='diagnosis', ascending=False), annot=True, cmap='coolwarm', fmt=".2f", vmin=-1, vmax=1)
plt.title("Correlation of Training Features with Diagnosis (1=Malignant)")
plt.show()

# Print the most correlated features
sorted_corr_train = corr_matrix_train["diagnosis"].sort_values(ascending=False)
print("\nTop Positive Correlations with Diagnosis (Training Data):")
print(sorted_corr_train.head(10))
print("\nTop Negative Correlations (actually least positive here):")
print(sorted_corr_train.tail(5))

Features like `worst concave points`, `worst perimeter`, `worst radius`, `mean concave points` show strong positive correlation with malignancy (diagnosis=1). This makes intuitive sense.

Let's look at a scatter matrix for a few highly correlated features.

In [ ]:
from pandas.plotting import scatter_matrix # Already imported, but good to remember

# Select top features based on correlation with diagnosis (from training data)
# Exclude 'diagnosis' itself from the list of features to plot against each other
top_features_for_pairplot = [f for f in sorted_corr_train.index if f != 'diagnosis'][:4]

print(f"\nTop features selected for PairPlot: {top_features_for_pairplot}")

if top_features_for_pairplot: # Proceed only if features were selected
    # sns.pairplot creates its own figure, so plt.figure might not be needed or could interfere.
    sns.pairplot(X_train_with_target, vars=top_features_for_pairplot, hue='diagnosis',
                 palette={0: 'skyblue', 1: 'salmon'}, corner=True)
    # Adjust title positioning if using sns.pairplot's figure
    plt.gcf().suptitle(f"Pairplot of Top {len(top_features_for_pairplot)} Correlated Features (Training Data)", y=1.03)
    plt.show()
else:
    print("No features selected for pairplot based on correlation, skipping.")

In the scatter matrix, look at the plots for pairs of features. You're hoping to see that malignant and benign points separate out.

### **Box Plots by Diagnosis (Training Data)**

Comparing feature distributions for benign vs. malignant cases using box plots on the training data.

In [ ]:
# Select a subset of features for clearer boxplots.
if 'top_features_for_pairplot' in locals() and top_features_for_pairplot and len(top_features_for_pairplot) > 0 :
    # Take distinct features, ensuring they exist in X_train_with_target
    base_features = [f for f in top_features_for_pairplot if f in X_train_with_target.columns]
    additional_features = [f for f in ['mean texture', 'mean area', 'mean smoothness'] if f in X_train_with_target.columns and f not in base_features]
    features_for_boxplot_train = (base_features + additional_features)[:6] # Cap at 6 features for readability
else:
    # Fallback if top_features_for_pairplot wasn't properly defined or is empty
    features_for_boxplot_train = ['mean radius', 'mean texture', 'mean perimeter', 'mean area', 'mean smoothness', 'mean concavity']
    features_for_boxplot_train = [f for f in features_for_boxplot_train if f in X_train_with_target.columns][:6] # Ensure they exist and cap

if not features_for_boxplot_train: # Ultimate fallback if list is still empty
    print("Error: No valid features selected for boxplot. Using minimal default example.")
    features_for_boxplot_train = [col for col in ['mean radius', 'mean texture'] if col in X_train_with_target.columns]


n_box_features = len(features_for_boxplot_train)
if n_box_features > 0: # Proceed only if there are features to plot
    n_box_cols = min(3, n_box_features) # Max 3 columns for boxplots
    n_box_rows = (n_box_features + n_box_cols - 1) // n_box_cols # Calculate rows needed

    plt.figure(figsize=(n_box_cols * 5, n_box_rows * 4)) # Setup figure size
    for i, feature in enumerate(features_for_boxplot_train):
        plt.subplot(n_box_rows, n_box_cols, i + 1) # Create subplot for each feature
        # sns.boxplot shows distribution summaries.
        # x='diagnosis' groups data by diagnosis.
        # y=feature specifies the feature to plot.
        # hue='diagnosis' and palette provide coloring. legend=False as x-axis is clear.
        sns.boxplot(x='diagnosis', y=feature, data=X_train_with_target,
                    hue='diagnosis', palette={0: 'skyblue', 1: 'salmon'}, legend=False)
        plt.title(feature) # Title for each subplot

    plt.suptitle("Boxplots of Selected Features by Diagnosis (Training Data: 0=Benign, 1=Malignant)", fontsize=16, y=1.02) # Overall title
    plt.tight_layout(rect=[0, 0, 1, 0.97]) # Adjust layout to prevent overlap
    plt.show()
else:
    print("No features available for boxplotting.")

This plot should show some decent separation, with malignant tumors generally having higher values.

**Experiment with Attribute Combinations** (Feature Engineering)

Sometimes, creating new features from existing ones can help. For this dataset, the features are already quite meaningful. But for illustration, if we had `total_area` and `num_cells`, a `area_per_cell` might be useful.
For now, we'll stick with the original features as they are well-established for this problem.

---
# 5. Prepare the Data for Machine Learning Algorithms

Time to get the data ready! We need to handle:
*   Missing values (our dataset is clean, but we'll show how).
*   Categorical features (we don't have any, but we'll show how).
*   Feature Scaling (very important!).

We'll use Scikit-Learn's `Pipeline` to make this process neat and reproducible.

### **Handling Missing Values (Illustrative)**
Our dataset doesn't have missing values. But if it did, `SimpleImputer` is a good tool.

In [ ]:
from sklearn.impute import SimpleImputer # For handling missing values

# Create a temporary copy of X_train and introduce some NaNs for demonstration
X_train_temp_for_imputation = X_train.copy()
# Introduce some NaNs into the 'mean texture' column for the first 5 samples
for i in range(min(5, len(X_train_temp_for_imputation))): # Ensure we don't go out of bounds
    X_train_temp_for_imputation.iloc[i, X_train_temp_for_imputation.columns.get_loc('mean texture')] = np.nan

print("NaNs in 'mean texture' (temp df) before imputation:", X_train_temp_for_imputation['mean texture'].isnull().sum())

# Initialize SimpleImputer with median strategy
imputer = SimpleImputer(strategy="median")

# Fit the imputer ONLY on the column(s) with NaNs from the training data copy
imputer.fit(X_train_temp_for_imputation[['mean texture']])
# Transform the column(s), replacing NaNs with the learned median
X_train_temp_for_imputation['mean texture'] = imputer.transform(X_train_temp_for_imputation[['mean texture']])

print("NaNs in 'mean texture' (temp df) AFTER imputation:", X_train_temp_for_imputation['mean texture'].isnull().sum())
print("First 5 'mean texture' values (temp df) after imputation with median:")
print(X_train_temp_for_imputation['mean texture'].head())
print("\n(End of illustrative imputation. We'll use the original clean X_train going forward.)")

### **Handling Categorical Attributes (Illustrative)**

We don't have categorical features here. But if we did, like `tumor_grade` ('G1', 'G2', 'G3'), we'd use `OneHotEncoder`.

In [ ]:
from sklearn.preprocessing import OneHotEncoder # For converting categorical features to numerical

# Create dummy categorical data for illustration
# Make sure the index aligns if you were to merge this back, using X_train.index
if len(X_train) >= 5:
    X_train_cat_example = pd.DataFrame({'tumor_grade': ['G1', 'G2', 'G3', 'G2', 'G1']},
                                     index=X_train.index[:5])

    # Initialize OneHotEncoder
    # sparse_output=False returns a dense NumPy array.
    # handle_unknown='ignore' makes the encoder robust to new categories in test data.
    cat_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    # Fit and transform the dummy categorical data
    tumor_grade_1hot = cat_encoder.fit_transform(X_train_cat_example[['tumor_grade']])

    print("\nOne-hot encoding example for 'tumor_grade':")
    print(tumor_grade_1hot)
    print(f"Categories found by OneHotEncoder: {cat_encoder.categories_}")
    # get_feature_names_out is the modern way to get output feature names
    print(f"Feature names out from OneHotEncoder: {cat_encoder.get_feature_names_out(['tumor_grade'])}")
else:
    print("\nSkipping illustrative OneHotEncoding as X_train has less than 5 samples.")
print("\n(End of illustrative one-hot encoding.)")

### **Feature Scaling**

*Most* ML algorithms work better when numerical features are on a similar scale. We'll use `StandardScaler`.

In [ ]:
from sklearn.preprocessing import StandardScaler # For standardizing features (mean=0, std=1)
from sklearn.pipeline import Pipeline # For chaining preprocessing steps and models

# We will define the scaler as the first step within each model's pipeline later.
# This ensures that scaling is learned only from training data within cross-validation folds.
# For demonstration, let's apply it to X_train to see its effect.
num_pipeline_example = Pipeline([
    ('std_scaler', StandardScaler()),
])
X_train_prepared_np_example = num_pipeline_example.fit_transform(X_train)

print(f"\nShape of X_train after example scaling: {X_train_prepared_np_example.shape}")
print("A few values from X_train after example scaling (first 2 rows, first 5 cols):")
print(X_train_prepared_np_example[:2, :5])

# Convert back to DataFrame to check mean and std
X_train_prepared_df_example = pd.DataFrame(X_train_prepared_np_example, columns=X_train.columns, index=X_train.index)
print("\nMean of 'mean radius' after example scaling:", X_train_prepared_df_example['mean radius'].mean())
print("Std dev of 'mean radius' after example scaling:", X_train_prepared_df_example['mean radius'].std())
print("\n(Actual scaling will happen inside model pipelines on the fly.)")

---
# 6. Select and Train a Model

Now we select and train a few different classification models.

Each model will be part of a pipeline that includes scaling as the first step.

### **Baseline Model: Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression # Logistic Regression classifier
from sklearn.metrics import accuracy_score, roc_auc_score # Evaluation metrics

# Define the pipeline: 1. Scale data, 2. Apply Logistic Regression
log_reg_pipeline = Pipeline([
    ("scaler", StandardScaler()), # First step: standard scaler
    ("log_reg", LogisticRegression(solver="liblinear", random_state=42)) # Second step: logistic regression model
])
# Fit the pipeline on the training data
log_reg_pipeline.fit(X_train, y_train) # Scaler is fit_transformed on X_train, then model is fit

# Evaluate on the TRAINING set (for an initial feel - cross-validation is more robust)
y_train_pred_log_reg = log_reg_pipeline.predict(X_train) # Predict class labels
train_accuracy_log_reg = accuracy_score(y_train, y_train_pred_log_reg) # Calculate accuracy
train_probas_log_reg = log_reg_pipeline.predict_proba(X_train)[:, 1] # Probabilities for the positive class (Malignant=1)
train_auc_log_reg = roc_auc_score(y_train, train_probas_log_reg) # Calculate ROC AUC score
print(f"\nLogistic Regression TRAINING accuracy: {train_accuracy_log_reg:.4f}")
print(f"Logistic Regression TRAINING AUC: {train_auc_log_reg:.4f}")

High 90s for accuracy and AUC on training data is pretty good for a start!

### **Decision Tree Classifier**

In [ ]:
from sklearn.tree import DecisionTreeClassifier # Decision Tree classifier
tree_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("tree_clf", DecisionTreeClassifier(random_state=42)) # random_state for reproducibility
])
tree_pipeline.fit(X_train, y_train)

# Evaluate on the TRAINING set
y_train_pred_tree = tree_pipeline.predict(X_train)
train_accuracy_tree = accuracy_score(y_train, y_train_pred_tree)
train_probas_tree = tree_pipeline.predict_proba(X_train)[:, 1]
train_auc_tree = roc_auc_score(y_train, train_probas_tree)
print(f"\nDecision Tree TRAINING accuracy: {train_accuracy_tree:.4f}") # Often 1.0 if unconstrained (overfitting)
print(f"Decision Tree TRAINING AUC: {train_auc_tree:.4f}") # Often 1.0 if unconstrained

A perfect score on training data for the Decision Tree is a big sign of overfitting!

### **Better Evaluation Using Cross-Validation**
Cross-validation gives a more robust estimate of model performance on unseen data.

In [ ]:
from sklearn.model_selection import cross_val_score # Function for cross-validation

# Cross-validation for Logistic Regression pipeline
# cv=10 means 10-fold cross-validation. scoring="roc_auc" specifies the metric.
log_reg_auc_scores_cv = cross_val_score(log_reg_pipeline, X_train, y_train, cv=10, scoring="roc_auc")
print(f"\nLogistic Regression 10-fold CV AUC scores: {log_reg_auc_scores_cv.round(4)}")
print(f"Mean CV AUC: {log_reg_auc_scores_cv.mean():.4f}, Std CV AUC: {log_reg_auc_scores_cv.std():.4f}")

# Cross-validation for Decision Tree pipeline
tree_auc_scores_cv = cross_val_score(tree_pipeline, X_train, y_train, cv=10, scoring="roc_auc")
print(f"\nDecision Tree 10-fold CV AUC scores: {tree_auc_scores_cv.round(4)}")
print(f"Mean CV AUC: {tree_auc_scores_cv.mean():.4f}, Std CV AUC: {tree_auc_scores_cv.std():.4f}")
# Decision Tree CV score is usually much lower than its training score, indicating overfitting.

The Decision Tree's cross-validation AUC will be much lower than its training AUC, confirming overfitting. Logistic Regression's CV score is likely more stable and realistic.

### **Random Forest Classifier**
Random Forest is an ensemble model (multiple decision trees) that often performs well and is more robust to overfitting.

In [ ]:
from sklearn.ensemble import RandomForestClassifier # Random Forest classifier
forest_pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("forest_clf", RandomForestClassifier(n_estimators=100, random_state=42)) # n_estimators = number of trees
])
forest_auc_scores_cv = cross_val_score(forest_pipeline, X_train, y_train, cv=10, scoring="roc_auc")
print(f"\nRandom Forest 10-fold CV AUC scores: {forest_auc_scores_cv.round(4)}")
print(f"Mean CV AUC: {forest_auc_scores_cv.mean():.4f}, Std CV AUC: {forest_auc_scores_cv.std():.4f}")

---
# 7. Fine-Tune Your Model

Let's assume Random Forest is our most promising model. Now we fine-tune its hyperparameters.

**Grid Search (`GridSearchCV`)**
Tries all combinations of hyperparameters you specify.

In [ ]:
from sklearn.model_selection import GridSearchCV # For hyperparameter tuning

# Define the parameter grid for RandomForestClassifier within our pipeline
# Parameters are referred to by 'stepname__parametername'
param_grid_rf = [
    {'forest_clf__n_estimators': [50, 100, 150],        # Number of trees
     'forest_clf__max_features': ['sqrt', 'log2', None], # Max features to consider for a split ('auto' is teofn sqrt)
     'forest_clf__max_depth': [None, 10, 20],           # Max depth of each tree (None=unlimited)
     'forest_clf__min_samples_split': [2, 5, 10]}         # Min samples to split an internal node
]

# Use the same pipeline structure; GridSearchCV will set the parameters
# It's good practice to define it clearly here for the search.
forest_pipeline_for_grid = Pipeline([
    ("scaler", StandardScaler()),
    ("forest_clf", RandomForestClassifier(random_state=42)) # Base model with random_state for forest
])

# Initialize GridSearchCV
# cv=5: 5-fold cross-validation for each parameter combination
# scoring='roc_auc': Evaluate using ROC AUC
# return_train_score=True: Useful for diagnosing overfitting of specific param sets
# verbose=1: Shows progress messages
# n_jobs=-1: Use all available CPU cores to speed up the search
grid_search_rf = GridSearchCV(forest_pipeline_for_grid, param_grid_rf,
                              cv=5, scoring='roc_auc',
                              return_train_score=True, verbose=1, n_jobs=-1)
# Fit GridSearchCV on the training data (this can take some time)
grid_search_rf.fit(X_train, y_train)

print("\nGridSearchCV Best Parameters (Random Forest):")
print(grid_search_rf.best_params_) # The combination of parameters that gave the best CV score
print(f"Best CV AUC score from GridSearchCV: {grid_search_rf.best_score_:.4f}") # The best CV score

# The best_estimator_ attribute holds the pipeline fitted with the best parameters on the whole X_train
best_rf_model_gs = grid_search_rf.best_estimator_

GridSearchCV can take time. If the hyperparameter space is very large, `RandomizedSearchCV` is often a better bet. It samples a fixed number of hyperparameter combinations from specified distributions.

**(Optional) Randomized Search (`RandomizedSearchCV`)**

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs_rf = {
    'forest_clf__n_estimators': randint(low=50, high=250),
    'forest_clf__max_features': ['sqrt', 'log2', None],
    'forest_clf__max_depth': randint(low=5, high=35),
    'forest_clf__min_samples_split': randint(low=2, high=11),
    'forest_clf__min_samples_leaf': randint(low=1, high=11)
}

# Using the same forest_pipeline structure
# Set n_jobs=-1 to use all available CPU cores for parallel processing, which can speed things up.
# However, for full reproducibility across different machines/setups, n_jobs=1 might be preferred
# if there are subtle issues with how random states are handled in parallel by underlying libraries.
rnd_search_rf = RandomizedSearchCV(forest_pipeline,
                                   param_distributions=param_distribs_rf,
                                   n_iter=30, # Number of parameter settings that are sampled
                                   cv=5, scoring='roc_auc',
                                   random_state=42, verbose=1, return_train_score=True, n_jobs=-1)
# To run RandomizedSearchCV:
# rnd_search_rf.fit(X_train, y_train)
# print("\nRandomizedSearchCV Best Parameters (Random Forest):")
# print(rnd_search_rf.best_params_)
# print(f"Best CV AUC score from RandomizedSearchCV: {rnd_search_rf.best_score_:.4f}")
# best_rf_model_rs = rnd_search_rf.best_estimator_ # Update best model if you run this

For this demo, we'll proceed with the model from `GridSearchCV` (`best_rf_model_gs`).

### **Analyzing the Best Models and Their Errors**
Once you have your best model, you can inspect it. For Random Forests, feature importances tell you which features were most influential.

In [ ]:
# Let's use the best model from GridSearchCV
final_model_pipeline = best_rf_model_gs # or best_rf_model_rs if you ran RandomizedSearch and it was better

# The RandomForestClassifier is the step named "forest_clf" in our pipeline
final_model_classifier_step = final_model_pipeline.named_steps["forest_clf"]
feature_importances = final_model_classifier_step.feature_importances_

# Get feature names (they are the same as X_train.columns because scaler doesn't change them)
feature_names = X_train.columns

sorted_importances_with_names = sorted(zip(feature_importances, feature_names), reverse=True)

print("\nTop 10 Feature Importances (from best Random Forest):")
for importance, name in sorted_importances_with_names[:10]:
    print(f"{name}: {importance:.4f}")

plt.figure(figsize=(10, 8))
top_n = 15
# Plot horizontal bar chart (easier to read with many features)
# We take top_n and then reverse the list for plotting so the most important is at the top
names_for_plot = [name for imp, name in sorted_importances_with_names[:top_n]][::-1]
importances_for_plot = [imp for imp, name in sorted_importances_with_names[:top_n]][::-1]

plt.barh(names_for_plot, importances_for_plot)
plt.xlabel("Feature Importance")
plt.title(f"Top {top_n} Feature Importances")
plt.tight_layout() # Adjust layout to make sure everything fits
plt.show()

This can give biological insights! For example, 'worst concave points' or 'worst radius' might show up as highly important.

---
# 8. Evaluate Your System on the Test Set

The moment of truth! We evaluate our final, tuned model (`final_model_pipeline`) on the `X_test` and `y_test` data that we set aside at the very beginning.

In [ ]:
y_test_pred = final_model_pipeline.predict(X_test)
y_test_proba = final_model_pipeline.predict_proba(X_test)[:, 1] # Probabilities for the positive class (Malignant=1)

test_accuracy = accuracy_score(y_test, y_test_pred)
test_auc = roc_auc_score(y_test, y_test_proba)

print(f"\nFinal Model Performance on Test Set:")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test AUC: {test_auc:.4f}")

# You can also look at a confusion matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve

conf_matrix = confusion_matrix(y_test, y_test_pred)
print("\nConfusion Matrix on Test Set (TN, FP / FN, TP):")
print(conf_matrix)

disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=['Benign (0)', 'Malignant (1)'])
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix - Test Set")
plt.show()

# And plot the ROC curve for the test set
fpr, tpr, thresholds = roc_curve(y_test, y_test_proba)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {test_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) - Test Set')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

The test set performance is your best estimate of how your model will perform on new, unseen data. Resist the temptation to tweak your model further based on these test set results!

---
# 9. Launch, Monitor, and Maintain Your System (Briefly)

If this were a real-world application:

**Save Your Model**: Use `joblib` to save your trained pipeline.

In [ ]:
import joblib

model_filename = "breast_cancer_classification_model.pkl"
joblib.dump(final_model_pipeline, model_filename)
print(f"\nModel saved as {model_filename}")

# To load it later:
# loaded_model = joblib.load(model_filename)
# predictions = loaded_model.predict(new_data)

*   **Deploy**: This could mean wrapping it in a web service, integrating it into lab software, etc.
*   **Monitor**: Continuously check its performance on new data. Monitor for data drift or concept drift. For medical uses, alerts for changes in error rates (especially false negatives) are critical.
*   **Maintain**: Regularly retrain your model on fresh data. Keep backups of datasets and model versions.

---
And that's a walkthrough of an end-to-end machine learning project. The key is a systematic approach.